# 비정형 빅데이터 응용과 실습 - Week 07
Elasticsearch와 mysql을 결합한 검색엔진을 만들어보겠습니다.
____

## 1. 데이터 로드

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("movie_doc.csv",sep="\t")

In [ ]:
df.head()

## 2. MySQL에 데이터 적재

In [ ]:
import mysql.connector

HOST = ""
USER = ""
PASSWORD = ""
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD
)

mycursor = mydb.cursor()
# mycursor.execute("DROP DATABASE mydb")
try:
    mycursor.execute("CREATE DATABASE movie_db")
except:
    pass

In [ ]:
mydb = mysql.connector.connect(
  host=HOST,
  user=USER,
  password=PASSWORD,
  database="movie_db"
)

mycursor = mydb.cursor()
try:
    mycursor.execute("DROP TABLE movie;")
except:
    pass
mycursor.execute("""
                    CREATE TABLE movie 
                    (id VARCHAR(255) UNIQUE, story TEXT, title VARCHAR(255), link VARCHAR(255), image VARCHAR(255))"""
                )


In [ ]:
for row_dict in df.to_dict(orient="records"):
    print(row_dict)

In [ ]:
for row_dict in df.to_dict(orient="records"):
    print(row_dict)
    movie_id = str(row_dict['id'])
    title = row_dict['title']
    story = row_dict['story']
    link = row_dict['link']
    image = row_dict['image']

    sql = "INSERT INTO movie (id, story, title, link, image) VALUES (%s, %s, %s, %s, %s)"
    val = (movie_id, story, title, link, image)
    mycursor.execute(sql, val)

    mydb.commit()

    print(mycursor.rowcount, "record inserted.")

In [ ]:
sql = "SELECT * FROM movie"
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)

## 3. ElasticSearch에 데이터 인덱싱

In [ ]:
from elasticsearch import Elasticsearch, helpers


In [ ]:
import pprint  
INDEX_NAME = "movie_index"


INDEX_SETTINGS = {
  "settings" : {
    "index":{
      "analysis":{
        "analyzer":{
          "korean":{
            "type":"custom",
            "tokenizer":"seunjeon_tokenizer"
          }
        }
      }
    }
  },
  "mappings": {

      "properties" : {
        "story" : {
          "type" : "text",
          "analyzer": "korean"
        },
        "title" : {
          "type" : "text",
          "analyzer": "korean"
        }
      }

  }
}


In [ ]:
user = ""
password = ""
host = ""

try:
    es.transport.close()
except:
    pass
es = Elasticsearch([host],http_auth=(user,password),scheme="https",port=443)



In [ ]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

In [ ]:
movies = [
    {
        "_index": INDEX_NAME,
        "_id" : doc['id'],
        "_source": {
            "title": doc['title'],
            "story": doc['story'],
        }
    }
    for doc in myresult
]

movies[:10]

In [ ]:
try:
    response = helpers.bulk(es, movies)
    print ("\nRESPONSE:", response)
except Exception as e:
    print("\nERROR:", e)

In [ ]:
doc = es.get(index=INDEX_NAME, id=1)
pprint.pprint(doc)

## 4. ElasticSearch에서 검색후, 검색결과를 MySQL에서 재 검색

In [ ]:
query="은퇴한 킬러"
res = es.search(index=INDEX_NAME, q=query)
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))
    print("Title: {}".format(hit['_source']['title']))
    print("줄거리: {}".format(hit['_source']['story']))


In [ ]:
movie_id = res['hits']['hits'][0]["_id"]

In [ ]:
sql = "SELECT * FROM movie WHERE id = {}".format(movie_id)
mycursor = mydb.cursor(dictionary=True)
mycursor.execute(sql)


myresult = mycursor.fetchall()
for x in myresult:
    print(x)